# Uses interaction to push triples to the brain

Before running, start GraphDB and make sure that there is a sandbox repository

In [1]:
# general imports for EMISSOR and the BRAIN
import emissor as em
from cltl import brain
from cltl.triple_extraction.api import Chat, UtteranceHypothesis
from emissor.persistence import ScenarioStorage
from emissor.representation.annotation import AnnotationType, Token, NER
from emissor.representation.container import Index
from emissor.representation.scenario import Modality, ImageSignal, TextSignal, Mention, Annotation, Scenario
from cltl.brain.long_term_memory import LongTermMemory
from cltl.combot.backend.api.discrete import UtteranceType
from cltl.brain.utils.helper_functions import brain_response_to_json
from cltl.reply_generation.lenka_replier import LenkaReplier

import os
import time
import uuid
from datetime import datetime
from datetime import date
import requests


/Users/piek/PycharmProjects/cltl-chatbots/venv/lib/python3.7/site-packages/rdflib_jsonld/__init__.py:12: DeprecationWarning: The rdflib-jsonld package has been integrated into rdflib as of rdflib==6.0.1.  Please remove rdflib-jsonld from your project's dependencies.
  DeprecationWarning,
[nltk_data] Downloading package punkt to /Users/piek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import sys
import os
src_path = os.path.abspath(os.path.join('..'))
if src_path not in sys.path:
    sys.path.append(src_path)

#### The next utils are needed for the interaction and creating triples and capsules
import util.driver_util as d_util
import util.capsule_util as c_util
import intentions.talk as talk

In [3]:
from random import getrandbits

##### Setting the location
place_id = getrandbits(8)
location = requests.get("https://ipinfo.io").json()

##### Setting the agents
AGENT = "Leolani2"
HUMAN_NAME = "Stranger"
HUMAN_ID = "stranger"

### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H:%M:%S")

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = "../../data"

### Define the folder where the images are saved
imagefolder = scenario_path + "/" + scenario_id + "/" + "image"


### Create the scenario folder, the json files and a scenarioStorage and scenario in memory
scenarioStorage = d_util.create_scenario(scenario_path, scenario_id)
scenario = scenarioStorage.create_scenario(scenario_id, datetime.now().microsecond, datetime.now().microsecond, AGENT)

Directory  ../../data/2021-11-01-20:59:11  Created 
Directory  ../../data/2021-11-01-20:59:11/image  Created 


In [4]:
import pathlib
log_path=pathlib.Path('./logs')
print(type(log_path))
my_brain = brain.LongTermMemory(address="http://localhost:7200/repositories/sandbox",
                           log_dir=log_path,
                           clear_all=True)
replier = LenkaReplier()

<class 'pathlib.PosixPath'>
2021-11-01 20:59:15,540 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Uploading ontology to brain
2021-11-01 20:59:17,437 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Computed trust for all known agents
2021-11-01 20:59:17,438 -    DEBUG -   cltl.reply_generation.api.LenkaReplier - Booted


In [ ]:
#### Initial prompt by the system from which we create a TextSignal and store it
initial_prompt = "Hi there. Who are you " + HUMAN_NAME + "?"
print(AGENT + ": " + initial_prompt)
textSignal = d_util.create_text_signal(scenario, initial_prompt)
scenario.append_signal(textSignal)

utterance = input('\n')
print(HUMAN_NAME + ": " + utterance)
while not (utterance.lower() == 'stop' or utterance.lower() == 'bye'):
    textSignal = d_util.create_text_signal(scenario, utterance)
    scenario.append_signal(textSignal)
    
    thoughts = talk.process_text_and_reply (scenario, place_id, location, HUMAN_ID, textSignal, replier, my_brain)
    
    print(AGENT + ": " + thoughts)
    textSignal = d_util.create_text_signal(scenario, thoughts)
    scenario.append_signal(textSignal)

    ###### Getting the next input signals
    utterance = input('\n')


Leolani2: Hi there. Who are you Stranger?



 I like cats


Stranger: I like cats
2021-11-01 20:59:32,197 -     INFO - cltl.triple_extraction.api.Chat (stranger)          000 - << Start of Chat with stranger >>
2021-11-01 20:59:32,198 -     INFO -               cltl.triple_extraction.api - Started POS tagger
2021-11-01 20:59:32,203 -     INFO -               cltl.triple_extraction.api - Started NER tagger
2021-11-01 20:59:32,206 -     INFO -               cltl.triple_extraction.api - Loaded grammar
2021-11-01 20:59:33,615 -     INFO - cltl.triple_extraction.api.Chat (stranger)          001 -   stranger: "I like cats"
{'subject': {'label': 'stranger', 'type': ['noun.person']}, 'predicate': {'label': 'like', 'type': ['verb.emotion']}, 'object': {'label': 'cats', 'type': ['noun.animal']}}
{'subject': {'label': 'stranger', 'type': 'person'}, 'predicate': {'label': 'like', 'type': 'emotion'}, 'object': {'label': 'cats', 'type': 'animal'}}
{'chat': '2021-11-01-20:59:11', 'turn': 'e1233d73-20e5-4d28-94ca-ed5f84053b13', 'author': 'stranger', 'utterance


 What do I like?


2021-11-01 20:59:58,994 -     INFO - cltl.triple_extraction.api.Chat (stranger)          000 - << Start of Chat with stranger >>
2021-11-01 20:59:59,887 -     INFO - cltl.triple_extraction.api.Chat (stranger)          001 -   stranger: "What do I like?"
{'predicate': {'label': 'like', 'type': ['verb.emotion']}, 'subject': {'label': 'stranger', 'type': ['noun.person']}, 'object': {'label': '', 'type': []}}
{'subject': {'label': 'stranger', 'type': 'person'}, 'predicate': {'label': 'like', 'type': 'emotion'}, 'object': {'label': '', 'type': []}}
{'chat': '2021-11-01-20:59:11', 'turn': '49f3a8f9-7e05-4833-acb1-f9d740a18590', 'author': 'stranger', 'utterance': 'What do I like?', 'utterance_type': <UtteranceType.QUESTION: 1>, 'position': '0-15', 'subject': {'label': 'stranger', 'type': 'person'}, 'predicate': {'label': 'like', 'type': 'emotion'}, 'object': {'label': '', 'type': []}, 'perspective': None, 'context_id': 'Leolani2', 'date': datetime.date(2021, 11, 1), 'place': 'Weesp', 'place_i

In [6]:
#scenario.scenario.end = datetime.now().microsecond
scenarioStorage.save_scenario(scenario)

## End of notebook

In [6]:
!pip list

Package                Version
---------------------- ------------
blis                   0.7.5
catalogue              2.0.6
certifi                2021.10.8
cffi                   1.15.0
chardet                3.0.4
charset-normalizer     2.0.7
click                  8.0.3
cltl.asr               0.0.dev3
cltl.backend           0.0.dev1
cltl.brain             0.0.dev6
cltl.reply-generation  0.0.dev5
cltl.triple-extraction 0.0.dev3
cltl.vad               0.0.dev1
cymem                  2.0.6
emissor                0.0.dev5
en-core-web-sm         3.1.0
filelock               3.3.2
Flask                  2.0.2
fuzzywuzzy             0.18.0
huggingface-hub        0.0.19
idna                   2.10
importlib-metadata     4.8.1
importlib-resources    5.3.0
iribaker               0.2
isodate                0.6.0
itsdangerous           2.0.1
Jinja2                 3.0.2
jiwer                  2.2.1
joblib                 1.1.0
jsonpickle             2.0.0
MarkupSafe             2.0.1
marshmall